In [19]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time

import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np

## Filtros

In [20]:
def filterImage (img, k):
    result = cv.GaussianBlur(img, (k,k), 0)
    return result

## Border detection

In [21]:
def borderDetectionImage (img, k):
    sobel_x = cv.Sobel(img, cv.CV_8UC1, dx=1, dy=0, ksize=k)
    sobel_y = cv.Sobel(img, cv.CV_8UC1, dx=0, dy=1, ksize=k)

    return sobel_x + sobel_y

## Corner detection

In [22]:
def cornerDetectionImage (img, blockSize, ksize, k):
    dst = cv.cornerHarris(img,blockSize,ksize,k)
    dst = cv.dilate(dst,None)

    return dst

## Treshold

In [23]:
def treshold(img, treshLevel):
    result = cv.adaptiveThreshold(img, 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, treshLevel, 3)
    return result

## Main

In [24]:
def processFrame (frame, gaussianFilterSize, sobelKSize, treshLevel, blockSize, harrisK, imageMode):
    ## Converte frame para tons de cinza
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    gray_3 = cv.cvtColor(gray, cv.COLOR_GRAY2BGR)  

    ## Filtros
    filtered = filterImage(gray, gaussianFilterSize)

    ## Passa pela detecção de bordas
    borderDetection = borderDetectionImage(filtered, sobelKSize)

    ## Filtra com um treshold
    resultTreshold = treshold(borderDetection, treshLevel)

    result = cv.cvtColor(resultTreshold, cv.COLOR_GRAY2BGR)    
    cornerDetection = cornerDetectionImage(resultTreshold, blockSize, sobelKSize, harrisK)
    result[cornerDetection>0.01*cornerDetection.max()]=[0,0,255]
    
    ## Exibe na saida saida
    if(imageMode):
        plt.imshow(result)
        plt.title('Saida')
        plt.show()
    else:
        imgOutput = np.concatenate((frame, result), axis=1)
        cv.imshow('VisaoComputacional', imgOutput)

## Parâmetros

Nessa primeira parte definimos alguns parâmetros para a execução do programa

In [25]:
VIDEO = cv.VideoCapture("./assets/tenis2.mp4")
ret, VIDEO_SAMPLE = VIDEO.read()
frame_rate = 30

# Valores default
gaussianFilterSize = 29
sobelKSize = 3
treshLevel = 7
blockSize = 3
harrisK = 0.21

### Calibração
Com base em uma imagem exemplo do vídeo podemos testar a melhor calibração e alterar

In [26]:
interact(processFrame, frame=fixed(VIDEO_SAMPLE), \
         gaussianFilterSize=widgets.IntSlider(min=1, max=255, step=2, value=gaussianFilterSize), \
         sobelKSize=widgets.IntSlider(min=1, max=31, step=2, value=sobelKSize), \
         treshLevel=widgets.IntSlider(min=1, max=30, step=2, value=treshLevel), \
         blockSize=widgets.IntSlider(min=1, max=30, step=1, value=blockSize), \
         harrisK=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=harrisK), \
         imageMode=fixed(True))

interactive(children=(IntSlider(value=29, description='gaussianFilterSize', max=255, min=1, step=2), IntSlider…

<function __main__.processFrame(frame, gaussianFilterSize, sobelKSize, treshLevel, blockSize, harrisK, imageMode)>

## Fluxo principal

Entra no loop principal, chama a função para processar a imagem

In [27]:
# Cria tela
cv.namedWindow('VisaoComputacional', cv.WINDOW_NORMAL)
cv.resizeWindow('VisaoComputacional', 800,600)

while (VIDEO.isOpened()):
    start_time = time.time()
    ret, frame = VIDEO.read()
    if not ret:
        print("Saindo...")
        break

    ## Chama função para processar frame
    processFrame(frame, gaussianFilterSize, sobelKSize, treshLevel, blockSize, harrisK, False)
    if cv.waitKey(1) == ord('q'):
        break

    time_elapsed = time.time() - start_time
    time_to_wait = 1./frame_rate - time_elapsed
    if time_to_wait > 0:
        time.sleep(time_to_wait)


VIDEO.release()
cv.destroyAllWindows()